In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import cv2
import os
import shutil

from glob import glob
from pathlib import Path

In [2]:
!pip install wolta

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 16.2 MB/s eta 0:00:00


In [4]:
os.environ['WANDB_MODE'] = 'disabled'

# Data Analysis

In [5]:
for dirname, _, _ in os.walk('/kaggle/input'):
    print(dirname)

/kaggle/input
/kaggle/input/test
/kaggle/input/test/FAKE
/kaggle/input/test/REAL
/kaggle/input/train
/kaggle/input/train/FAKE
/kaggle/input/train/REAL


In [6]:
p_paths = glob('/kaggle/input/*')
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))
print(d_paths)

['/kaggle/input/test/FAKE', '/kaggle/input/test/REAL', '/kaggle/input/train/FAKE', '/kaggle/input/train/REAL']


In [7]:
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

120000


In [8]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{'jpg': 120000}

In [9]:
from wolta.visual_tools import dataset_size_same

dataset_size_same(i_paths)

True

In [10]:
temp_img = cv2.imread(i_paths[0])
ratio = temp_img.shape[1] / temp_img.shape[0]

print('Width: {}'.format(temp_img.shape[1]))
print('Height: {}'.format(temp_img.shape[0]))
print('Ratio: {}'.format(ratio))

Width: 32
Height: 32
Ratio: 1.0


# Image Stacking

In [11]:
os.makedirs('/kaggle/working/raw')

In [12]:
for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = '/kaggle/working/raw/{}'.format(current_dir) 
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))

    for i_path in i_paths:
        shutil.copy(i_path, current_path)

# Image Splitting

In [13]:
from wolta.visual_tools import dir_split

dir_split('/kaggle/working/raw', '/kaggle/working/data', test_size=0.2, val_size=0.2)

In [14]:

from tqdm import tqdm

def add_gaussian_noise(image, mean=0, std=25):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

def process_and_save_images(input_path, output_path):
    """Process images, add noise, and save to the output path."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    image_paths = os.listdir(input_path)
    for img_name in tqdm(image_paths):
        img_path = os.path.join(input_path, img_name)
        if os.path.isfile(img_path):
            # Read image
            image = cv2.imread(img_path)
            if image is None:
                continue
            # Add Gaussian noise
            noisy_image = add_gaussian_noise(image)
            # Save the noisy image to the output folder
            img_name = img_name.split('.')[0] + '_1.' + img_name.split('.')[1]
            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, noisy_image)
base_dir = "/kaggle/working/data"  # Replace with the root path of your dataset
subsets = [ "test"]
categories = ["REAL", "FAKE"]

for subset in subsets:
    real_path = os.path.join(base_dir, subset, "REAL")
    fake_path = os.path.join(base_dir, subset, "FAKE")

    print(f"Processing {subset}/REAL...")
    process_and_save_images(real_path, real_path)
    print(f"Processing {subset}/FAKE...")
    process_and_save_images(fake_path, fake_path)


Processing test/REAL...


100%|██████████| 10000/10000 [00:02<00:00, 4475.34it/s]


Processing test/FAKE...


100%|██████████| 12000/12000 [00:02<00:00, 4306.02it/s]


In [15]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('/kaggle/working/data')

{'REAL': 60000, 'FAKE': 72000}

# YOLO

In [16]:
from ultralytics import YOLO

model = YOLO(model='yolo11x-cls.pt')
results = model.train(data='/kaggle/working/data', epochs=5, imgsz=32, verbose= True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 56.9M/56.9M [00:00<00:00, 161MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.pt, data=/kaggle/working/data, epochs=5, time=None, patience=100, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

2024-12-15 22:31:13,249	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-15 22:31:13,758	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

...

               classes   top1_acc   top5_acc: 100%|██████████| 688/688 [00:12<00:00, 54.67it/s]

                   all      0.903          1



5 epochs completed in 0.376 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 57.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 57.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 66000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 22000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 688/688 [00:10<00:00, 68.68it/s]


                   all      0.916          1
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [17]:
test_results = model.val(data='/kaggle/working/data', imgsz=32, split="test")

Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 66000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 22000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


test: Scanning /kaggle/working/data/test... 44000 images, 0 corrupt: 100%|██████████| 44000/44000 [00:19<00:00, 2202.35it/s]


test: New cache created: /kaggle/working/data/test.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 2750/2750 [00:31<00:00, 86.22it/s]


                   all      0.773          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [18]:
# Save the trained model
model.save('trained_yolo_model.pt')
shutil.rmtree('/kaggle/working/data')
shutil.rmtree('/kaggle/working/raw')